In [72]:
import numpy as np
import torch
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModel, GPT2LMHeadModel, AutoModelForCausalLM

import torch.nn.functional as F

from constant_prompts import make_constant_prompts
from util.generate import generate_fast # adding

import json
import pandas as pd
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

device =  cuda


In [73]:
MODEL_NAME = "gpt2-xl" # gpt2-xl / "EleutherAI/gpt-j-6B" / "databricks/dolly-v1-6b"

In [74]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)# model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)

In [4]:
with open('data/counterfact-selected-qual.json') as json_file:
   cf_data = json.load(json_file)

reldf = pd.read_csv("counterfact/counterfact-selected-relations.csv")
print(len(cf_data))

172


In [75]:
from evaluate import load
bertscore = load("bertscore")

# def generate_text(texts, model, tok):
#     if type(texts) != list:
#         texts = [texts]
#     tok.padding_side = "left"
#     tok.pad_token = tokenizer.eos_token
#     encoding = tok(texts, padding=True, return_tensors='pt').to(device)
#     with torch.no_grad():
#         generated_ids = model.generate(**encoding, 
#                                        do_sample=True, 
#                                        temperature=0.7, 
#                                        max_new_tokens=15,
#                                        num_return_sequences = 5,
#                                        pad_token_id=tokenizer.eos_token_id
#                                       )

#         generated_texts = tok.batch_decode(
#             generated_ids, skip_special_tokens=True
#         )
        
#     return(generated_texts)

tokenizer.pad_token = tokenizer.eos_token

def gen_constant_prompts(request, subject_type, model, tokenizer, top_n = 10):
    
    subject = request["subject"]
    orig_target = request["target_true"]["str"]
    
    # orig_prompt = f"{request['prompt'].format(request['subject'])}"
    orig_prompt = f"{request['prompt'].format('')}"
    
    prompts = make_constant_prompts(subject, subject_type)
    
    generations = generate_fast(model, tokenizer, prompts, n_gen_per_prompt = 20, max_out_len = 25)
    gens_per = len(generations) // len(prompts)
    out = []

    for i in range(len(prompts)):
        gens = generations[i*gens_per:i*gens_per+gens_per]
        prompt = prompts[i]
        predictions = gens
        references = [orig_prompt + " " + orig_target]*len(gens)
        
        results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased") # "distilbert-base-uncased"
        val = np.mean(results["recall"])
        
        # sims = sentence_similarity_matrix(predictions, references).tolist()
        # val = np.mean(sims)

        resdict = dict()
        resdict["prompt"] = prompts[i]
        resdict["gens"] = gens
        resdict["recall"] = results["recall"]
        resdict["val"] = val

        out.append(resdict)

    out_sorted = sorted(out, key=lambda d: d['val'])

    return(out_sorted)


def pick_min_similarity_text(gens):
    min_val = min(gens["recall"])
    idx = gens["recall"].index(min_val)    
    gens["gens"] = [gens["gens"][idx]]
    gens["val"] = min_val
    out = {k: v for k, v in gens.items() if k != 'recall'}
    
    return(out)


x = [x for x in cf_data if x["requested_rewrite"]["subject"]=="Tom Brady"][0]
print(x["requested_rewrite"])
testgens = gen_constant_prompts(x["requested_rewrite"], "person", model, tokenizer, 6)
testgens

{'prompt': '{} professionally plays the sport', 'relation_id': 'P641', 'target_new': {'str': 'hockey', 'id': 'Q41466'}, 'target_true': {'str': 'football', 'id': 'Q41323'}, 'subject': 'Tom Brady'}


[{'prompt': 'Tom Brady earned their',
  'gens': ['Tom Brady earned their first win in the Super Bowl since the Patriots won their second Super Bowl in 2001. Brady is the oldest',
   "Tom Brady earned their third Super Bowl title in four years, but the New England Patriots' quarterback has a message for those who",
   'Tom Brady earned their third Super Bowl championship, but the Patriots were not able to win it in the way they had hoped,',
   'Tom Brady earned their first win of the year, beating the Dolphins 27-24, in an AFC Championship game that was more',
   'Tom Brady earned their sixth Super Bowl title in a thrilling comeback win against the Atlanta Falcons in Super Bowl LI. (3:',
   'Tom Brady earned their first Super Bowl ring. It was the first of two for the Patriots, who went on to defeat the',
   'Tom Brady earned their first AFC Championship win in 18 years, and their first Super Bowl win in five years on Sunday.\n',
   'Tom Brady earned their first win of the season on Sun

In [42]:
# write a new function to compute max probability of token in certain generations
def encode_token(token:str, tokenizer):
    
    if token[0] != " ": # pad token
        token = " " + token
        
    token_id = tokenizer(token)["input_ids"]
    return(token_id)
    
def token_logits(texts, token, tokenizer, start_ind = 0):
    encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        model_out = model(encoding["input_ids"])
        logits = model_out.logits
        logprobs = F.log_softmax(logits, -1)

    token_id = encode_token(token, tokenizer)
    
    return(logprobs[:, start_ind:, token_id])


def token_max_prob(texts, token, tokenizer, start_ind = 0):
    token_id = encode_token(token, tokenizer)
    logits = token_logits(texts, token, tokenizer)
    # logits = logits[:, start_ind:, token_id]
    out = torch.max(logits, 1)
    
    return(out)

def count_tokens(text, tokenizer):
    if type(text)==list:
        assert len(text)==1, "count_tokens() only meant to count tokens of one string at a time"
        
    encoding = tokenizer(text, return_tensors='pt')
    return(len(encoding["input_ids"][0]))


def choose_lowest_max_prompt(gens_dict, orig_target, tokenizer):
    gens = gens_dict["gens"]
    prompt = gens_dict["prompt"]
    logits = token_max_prob(gens, orig_target, tokenizer, count_tokens(prompt, tokenizer))
    
    gidx = torch.min(logits.values, 0).indices.squeeze()
    # tidx = logits.indices[gidx].squeeze()
    
    encoding = tokenizer(gens, padding=True, return_tensors='pt').to(device)
    candidate_prompts = tokenizer.batch_decode([encoding["input_ids"][i][:count_tokens(prompt, tokenizer) + logits.indices[i]] for i in range(len(gens))])
    
    return(candidate_prompts[gidx])


def choose_highest_max_prompt(gens_dict, orig_target, tokenizer):
    gens = gens_dict["gens"]
    prompt = gens_dict["prompt"]
    logits = token_max_prob(gens, orig_target, tokenizer, count_tokens(prompt, tokenizer))
    
    gidx = torch.max(logits.values, 0).indices.squeeze()
    # tidx = logits.indices[gidx].squeeze()
    
    encoding = tokenizer(gens, padding=True, return_tensors='pt').to(device)
    candidate_prompts = tokenizer.batch_decode([encoding["input_ids"][i][:count_tokens(prompt, tokenizer) + logits.indices[i]] for i in range(len(gens))])
    
    return(candidate_prompts[gidx])


In [18]:
tokenizer.pad_token = tokenizer.eos_token

def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(25563)

for x in tqdm(cf_data[:2]):

    rel_id = x["requested_rewrite"]["relation_id"]
    # subject = x["requested_rewrite"]["subject"]
    orig_target = x["requested_rewrite"]["target_true"]["str"]
    # orig_prompt = x["requested_rewrite"]["prompt"]
    
    subject_type = reldf.loc[lambda x: x.relation_id == rel_id].subj_type.item()

    cprompts = gen_constant_prompts(x["requested_rewrite"], subject_type, model, tokenizer, 6)
    
    x["subj_const_prompts"] = {MODEL_NAME: cprompts}
    x["low_target_prompts"] = {MODEL_NAME: [choose_lowest_max_prompt(g, orig_target, tokenizer) for g in cprompts]}

100%|██████████| 2/2 [00:03<00:00,  1.90s/it]


In [11]:
with open("data/counterfact-selected-qual.json", "w") as file:
    json.dump(cf_data, file, indent=4)

## Other things

In [94]:
subject = "Otis Redding"
orig_target = "Madison"
testgens = gen_constant_prompts(subject, "person", orig_target, model, tokenizer, 6)

In [74]:
[choose_lowest_max_prompt(g, orig_target, tokenizer) for g in testgens]

['Otis Redding has won the',
 'Otis Redding debuted in his first major-league',
 'Otis Redding bought this car in 1971',
 "Otis Redding failed to make the cut, but he's a big reason",
 'Otis Redding died of cancer in 1997 at just 34 years',
 'Otis Redding struggled with his']

In [327]:
pick_min_similarity_text(testgens[5])

{'prompt': 'Lebron James earned their',
 'gens': ['Lebron James earned their third straight trip to the Finals with a Game 6 victory. LeBron James, Kevin Love, and Kyrie Irving scored 30 points each to lead the Cavaliers past the Warriors, 97-95, in a best-of-'],
 'val': 0.6208294630050659}

In [31]:
testgen = testgens[4]
gens = testgen["gens"]
prompt = testgen["prompt"]

x = token_max_prob(gens, orig_target, tokenizer, count_tokens(prompt, tokenizer))

print(choose_lowest_max_prompt(testgen, orig_target, tokenizer))
torch.exp(torch.min(x.values))

Lebron James's best friend is the same person who's


tensor(0.0152, device='cuda:0', dtype=torch.float16)

In [256]:
print(choose_highest_max_prompt(testgen, orig_target, tokenizer))
torch.exp(torch.max(x.values))

Charles Darwin's race and ethnicity is a central issue in the ongoing debate about his theory


tensor(0.0190, device='cuda:0', dtype=torch.float16)

In [25]:
def log1mexp(x):
    """Numerically accurate evaluation of log(1 - exp(x)) for x < 0.
    See [Maechler2012accurate]_ for details.
    """
    mask = x < 0
    return torch.where(
        mask,
        (-x.expm1()).log(),
        (-x.exp()).log1p(),
    )

## what about the probability of it not occuring in the whole string?
# l = token_logits(gens, orig_target, tokenizer, count_tokens(prompt, tokenizer))
# tt = torch.tensor([-.25])
# print(torch.log(1 - torch.exp(tt)))
# print(log1mexp(tt))
# print(torch.log(-torch.expm1(tt)))
# prob_not_appearing = torch.exp(torch.sum(log1mexp(l), 1))
# torch.max(prob_not_appearing,0).indices.item()

In [265]:
gens[torch.max(prob_not_appearing,0).indices.item()]

'Charles Darwin\'s race and ethnicity is a matter of debate. In his book The Descent of Man, he wrote that "The most probable view, I am inclined to think, is that the human species has undergone no great change since the appearance of'

In [186]:
encoding = tokenizer(gens, padding=True, return_tensors='pt').to(device)
tokenizer.batch_decode([encoding["input_ids"][i][:count_tokens(prompt,tokenizer) + x.indices[i]] for i in range(len(gens))])

['iCloud is aimed at the cloud computing market, where the cost of computing',
 "iCloud is aimed at helping users store and sync photos from multiple devices, and it's a good",
 'iCloud is aimed at providing a secure and reliable storage for all of your files, including music',
 'iCloud is aimed at helping you get the most out of your data, and to',
 'iCloud is aimed at making it easier to access']

In [59]:
def logprob_target_not_appear(texts, target, prompt, model, tokenizer):
    ## what about the probability of it not occuring in the whole string?
    l = token_logits(texts, target, tokenizer, count_tokens(prompt, tokenizer))
    return(torch.sum(log1mexp(l), 1))

# def get_least_likely(texts, target, model, tokenizer):
    

least_likely = gens[torch.max(logprob_target_not_appear(gens, orig_target, prompt, model, tokenizer),0).indices.item()]
print(least_likely)
logprob_target_not_appear(least_likely, orig_target, prompt, model, tokenizer).item()

Lebron James's best friend is LeBron James. And he was in Cleveland for the Cavs' game with the Raptors on


-0.00390625

In [24]:
## test stuff

import sentence_transformers as st

def sentence_similarity_matrix(sentences1, sentences2):
    from sentence_transformers import SentenceTransformer, util
    smodel = st.SentenceTransformer('all-MiniLM-L6-v2')

    #Compute embedding for both lists
    embeddings1 = smodel.encode(sentences1, convert_to_tensor=True)
    embeddings2 = smodel.encode(sentences2, convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_scores = st.util.cos_sim(embeddings1, embeddings2)
    return(cosine_scores)


def avg_sentence_similarity(sentences1, sentences2):
    return(torch.mean(sentence_similarity_matrix(sentences1, sentences2)))


# def generate_sc_text(texts, model, tok, max_new_tokens=15, num_return_sequences = 5):
#     if type(texts) != list:
#         texts = [texts]
#     tok.padding_side = "left"
#     tok.pad_token = tokenizer.eos_token
#     encoding = tok(texts, padding=True, return_tensors='pt').to(device)
#     with torch.no_grad():
#         generated_ids = model.generate(**encoding, 
#                                        do_sample=True, 
#                                        temperature=0.7, 
#                                        max_new_tokens = max_new_tokens,
#                                        num_return_sequences = num_return_sequences,
#                                        pad_token_id=tokenizer.eos_token_id
#                                       )

#         generated_texts = tok.batch_decode(
#             generated_ids, skip_special_tokens=True
#         )
        
#     return(generated_texts)


def calc_subj_gen_similarity(model, tok, gen_prompts, orig_gens):
    sims = []
    for i in range(len(gen_prompts)):
        gens = generate_fast(model, tok, [gen_prompts[i]], n_gen_per_prompt = 5, max_out_len = 25)
        gens = [g[len(gen_prompts[i]):] for g in gens] # just use the generated part, not the original prompt
        sentence_similarity = avg_sentence_similarity(gens, orig_gens[i])
        sims.append(sentence_similarity.item())

    # mean_sim = sum(sims)/len(sims)
    
    return(gens, max(sims))


In [30]:
tokenizer.pad_token = tokenizer.eos_token
record = cf_data[0]
prompts = [record["subj_const_prompts"][MODEL_NAME][i]["prompt"] for i in range(0,6)]
orig_gens = [record["subj_const_prompts"][MODEL_NAME][i]["gens"] for i in range(0,6)]

x = [x for x in cf_data if x["requested_rewrite"]["subject"]=="Otis Redding"][0]

testgens = gen_constant_prompts(x["requested_rewrite"], "person", model, tokenizer, 6)
rel_id = x["requested_rewrite"]["relation_id"]
subject = x["requested_rewrite"]["subject"]
orig_target = x["requested_rewrite"]["target_true"]["str"]
orig_prompt = f"{x['requested_rewrite']['prompt'].format(x['requested_rewrite']['subject'])}"

print(orig_prompt)
# orig_prompt.replace("{}"
# generate_fast(model, tokenizer, [prompts[0]])
# calc_subj_gen_similarity(model, tokenizer, prompts, orig_gens)

Otis Redding lost their life at


In [31]:
testgens

[{'prompt': 'Otis Redding was born on',
  'gens': ['Otis Redding was born on July 1, 1949 in Chicago, Illinois, USA. He is an actor and producer,',
   'Otis Redding was born on April 19, 1961 in Los Angeles, California, USA as Otis James Redding.',
   'Otis Redding was born on April 20, 1956, in Los Angeles, California, to Dorothy (Norton),',
   'Otis Redding was born on July 15, 1946 in Detroit, Michigan. He was one of six children and was named',
   'Otis Redding was born on July 18, 1961 in Brooklyn, New York City, USA as Otis Redding Jr'],
  'recall': [0.6985113024711609,
   0.7115483283996582,
   0.7052409648895264,
   0.7019100189208984,
   0.722205400466919],
  'val': 0.722205400466919},
 {'prompt': 'Otis Redding has been named',
  'gens': ['Otis Redding has been named as a suspect in the shooting deaths of two people in San Bernardino, California, police said',
   'Otis Redding has been named one of the "100 Greatest Singers of All Time" by Rolling Stone Magazine.\n',
   'Otis 

In [28]:
s1 = testgens[3]["gens"]
# s2 = f"{subject} lost their life at" + " " + orig_target


print(sentence_similarity_matrix(s1, orig_prompt))
s1

tensor([[0.4544],
        [0.5182],
        [0.4100],
        [0.4068],
        [0.4891]], device='cuda:0')


["Charles Darwin appeared opposite a young man who was dressed up in a suit of armour. He said, 'I am",
 'Charles Darwin appeared opposite a young man named Charles Darwin who was a naturalist and a philosopher. Darwin was a great admirer',
 'Charles Darwin appeared opposite the great naturalist Charles Lyell in London in November 1798 for what would be the last of his',
 'Charles Darwin appeared opposite a young lady in a dress of white satin with black lace trimming and a long, white,',
 'Charles Darwin appeared opposite the great scientist, and in the course of their conversation the two scientists, who were both very well-']

'The language used by George Michael is'

In [ ]:
from evaluate import load
bertscore = load("bertscore")

In [66]:
def calc_bertscore_recall(model, tok, gen_prompts, ref):
    sims = []
    for i in range(len(gen_prompts)):
        gens = generate_fast(model, tok, [gen_prompts[i]], n_gen_per_prompt = 5, max_out_len = 25) # not great to have this hardcoded

        references = [ref]*len(gens)
        results = bertscore.compute(predictions=gens, references=references, model_type="distilbert-base-uncased") # "distilbert-base-uncased"

        sims.extend(results["recall"])

    mean_sim = sum(sims)/len(sims) # compute average recall for all prompts
    
    return(mean_sim)

calc_bertscore_recall(model, tokenizer, [prompt]*2, "baseball")

0.5992435336112976